# Xây dựng mô hình dự đoán khách hàng rời mạng viến thông

Giữ chân khách hàng là yếu tố then chốt cho sự thành công của một công ty, đặc biệt trong một ngành cạnh tranh như dịch vụ viễn thông. Thu hút khách hàng mới không chỉ khó khăn hơn mà còn tốn kém hơn nhiều so với việc duy trì mối quan hệ với khách hàng hiện tại. Trong dự án này, chúng tôi sẽ dự đoán khách hàng có ý định rời bỏ dịch vụ tại một công ty cung cấp dịch vụ viễn thông. Trước tiên, chúng tôi sẽ sử dụng phân tích dữ liệu thăm dò để hiểu các mối quan hệ giữa các đặc điểm và biến mục tiêu, và xác định các yếu tố có ảnh hưởng trong quyết định rời bỏ của khách hàng. Sử dụng những đặc điểm này, chúng tôi sẽ phát triển một mô hình dự đoán để giúp công ty giảm tỷ lệ rời bỏ một cách chủ động và sử dụng những thông tin từ mô hình để củng cố các chiến lược giữ chân khách hàng.

## 1. Import thư viện

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Input

## 2. Load dữ liệu

In [ ]:
train_data = pd.read_csv('telecom_train.csv')
train_df = pd.DataFrame(data=train_data)

test_data = pd.read_csv('telecom_test.csv')
test_df = pd.DataFrame(data=test_data)

zip_cols = ['zip', 'density']
zip_data = pd.read_csv('uszips.csv', usecols=zip_cols)
zip_df = pd.DataFrame(data=zip_data)

In [ ]:
# show first 5 rows
pd.set_option('display.max_columns', None)
train_df.head(5)

## 3. Phân tích dữ liệu


In [ ]:
train_df.info()

Dữ liệu từ tập training bao gồm 5500 bản ghi với 38 thuộc tính được chia thành hai loại: dữ liệu nhân khẩu học của khách hàng và thông tin liên quan đến dịch vụ viễn thông của họ. Các đặc điểm nhân khẩu học bao gồm giới tính, tình trạng hôn nhân, số người phụ thuộc, và tuổi tác của khách hàng. Các đặc điểm liên quan đến thông tin tài khoản bao gồm thời gian khách hàng đã gắn bó với dịch vụ, chi phí hàng tháng và tổng chi phí, loại hợp đồng (theo tháng, một năm, hoặc hai năm), và loại dịch vụ điện thoại, internet, TV. Biến mục tiêu của chúng tôi cho nghiên cứu này là tình trạng khách hàng, một biến phân loại biểu thị liệu khách hàng ở lại hay đã rời dịch vụ.


In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.value_counts('Customer Status')

Số liệu cho thấy có 3663 người tiếp tục sử dụng dịch vụ, 1482 người đã ngừng sử dụng, và 355 người mới tham gia. Tỷ lệ giữ chân khách hàng khá tốt, với số người ở lại cao hơn đáng kể so với số người rời đi. Tuy nhiên, số lượng khách hàng mới còn thấp, chỉ bằng khoảng 1/4 số người rời đi.

### Phân tích đơn biến

In [ ]:
desc_df = train_df.describe(include=[object])
desc_df = desc_df.drop(columns=['Customer ID'])
desc_df

In [ ]:
train_df.describe(include=[np.number])


In [ ]:
train_df.hist(figsize=(15, 20), xrot=60)

Đồ thị này cung cấp một cái nhìn tổng quan về nhiều khía cạnh của dữ liệu khách hàng. Phân bố độ tuổi khá đều, với số lượng cao nhất ở nhóm 20-25. Đa số khách hàng không có người phụ thuộc. Số lượng giới thiệu thấp, với đa số khách hàng không giới thiệu ai. Thời gian sử dụng dịch vụ đa dạng, phần lớn khách hàng dùng từ 0 đến 20 tháng. Phí hàng tháng tập trung ở mức trung bình (phần đông trả 20 đến 25$ một tháng). Tổng chi phí càng cao thì số lượng khác hàng càng giảm. Phí dữ liệu phụ trội và cước gọi đường dài chủ yếu ở mức thấp với hầu hết khách hàng.

### Phân tích đa biến

In [ ]:
# encode customer status
class_dict = {'Stayed': 0, 'Churned': 1, 'Joined': 0}
train_df = train_df.replace({'Customer Status': class_dict})
test_df = test_df.replace({'Customer Status': class_dict})

In [ ]:
numeric_train_df = train_df.select_dtypes(include=[np.number])
fig, ax = plt.subplots(figsize=(15, 15))
axes = sns.heatmap(numeric_train_df.corr('spearman'), annot=True, ax=ax, linewidths=.5, fmt='.1f', cmap='rocket')

Tổng doanh thu có tương quan mạnh với tổng chi phí, thời gian sử dụng dịch vụ và cước gọi đường dài. Mã bưu điện, vĩ độ và kinh độ có mối liên hệ chặt chẽ, phản ánh vị trí địa lý. Thời gian sử dụng dịch vụ ảnh hưởng tích cực đến nhiều yếu tố như số lượng giới thiệu, chi phí hàng tháng và tổng chi phí. Thời gian ở lại tương quan âm với tình trạng khách hàng, chỉ ra rằng khách hàng càng có thâm niên thì càng ít có khả năng rời mạng.

### Trực quan hoá các thuộc tính có liên quan đến khả năng rời bỏ dịch vụ

In [ ]:
# scatterplot of total charge vs. monthly charge (with labels)
plt.figure(figsize=(10, 10))
sns.scatterplot(data=train_df, x='Monthly Charge', y='Total Charges', hue='Customer Status')

Khách hàng rời đi (màu cam) có xu hướng tập trung nhiều hơn ở vùng có phí hàng tháng cao hơn, đặc biệt là trong khoảng từ 60 đến 120. Khách hàng ở lại (màu xanh) phân bố đều hơn trên toàn bộ phạm vi phí hàng tháng, nhưng có vẻ chiếm ưu thế ở vùng phí thấp và trung bình. Tỷ lệ khách hàng rời đi có vẻ tăng lên khi phí hàng tháng tăng, thể hiện qua mật độ điểm màu cam cao hơn ở phía phải đồ thị.

In [ ]:
# scatter plot of tenure vs. total charges
plt.figure(figsize=(10, 10))
sns.scatterplot(data=train_df, x='Tenure in Months', y='Total Charges', hue='Customer Status')

Khách hàng ở lại (màu xanh) có xu hướng phân bố rộng hơn trên toàn bộ biểu đồ, cho thấy sự đa dạng trong thời gian gắn bó và tổng chi phí. Ngược lại, khách hàng rời đi (màu cam) tập trung nhiều hơn ở vùng có thời gian gắn bó ngắn hơn và tổng chi phí thấp hơn. Đáng chú ý là có một số nhóm điểm dữ liệu song song, cho thấy có thể có các gói dịch vụ hoặc mức giá khác nhau. Nhìn chung, khách hàng có thời gian gắn bó càng lâu thì tổng chi phí càng cao, nhưng nguy cơ rời đi có vẻ cao hơn trong giai đoạn đầu của dịch vụ.

In [ ]:
# boxplot of customer status vs. tenure
plt.figure(figsize=(10, 10))
sns.catplot(data=train_df, x='Customer Status', y='Tenure in Months', kind='box')

Khách hàng ở lại có thời gian gắn bó trung bình cao hơn (khoảng 37 tháng) và phạm vi rộng hơn (từ 15 đến 60 tháng). Ngược lại, khách hàng rời đi có thời gian gắn bó trung bình ngắn hơn nhiều (khoảng 10 tháng) và tập trung chủ yếu trong khoảng 3 đến 30 tháng. Điều này cho thấy khách hàng có khả năng rời đi cao nhất trong những tháng đầu. Tuy nhiên, vẫn có một số khách hàng rời đi sau thời gian dài gắn bó (thể hiện qua các điểm ngoại lệ).

In [ ]:
# barplot of gender vs count of customer status
plt.figure(figsize=(10, 10))
sns.catplot(data=train_df, x='Gender', hue='Customer Status', kind='count')

Số lượng khách hàng nam và nữ khá cân bằng, với nam giới có chút ít nhiều hơn. Tỷ lệ khách hàng rời đi so với ở lại cũng tương đối đồng đều giữa hai giới.

In [ ]:
# barplot of married vs count of customer status
plt.figure(figsize=(10, 10))
sns.catplot(data=train_df, x='Married', hue='Customer Status', kind='count')

Đối với khách hàng đã kết hôn, số lượng khách hàng ở lại (màu xanh) cao hơn đáng kể so với khách hàng rời đi (màu cam). Tuy nhiên, đối với khách hàng chưa kết hôn, sự chênh lệch này ít rõ rệt hơn, với tỷ lệ khách hàng rời đi cao hơn. Điều này gợi ý rằng khách hàng đã kết hôn có xu hướng trung thành hơn với dịch vụ. 

In [ ]:
# barplot of contact vs count of customer status
plt.figure(figsize=(10, 10))
sns.catplot(data=train_df, x='Contract', hue='Customer Status', kind='count')

Đối với hợp đồng Month-to-Month, tỷ lệ khách hàng rời đi (màu cam) cao nhất và gần bằng số khách hàng ở lại (màu xanh), cho thấy đây là nhóm có nguy cơ mất khách hàng cao nhất. Ngược lại, hợp đồng One Year và Two Year có tỷ lệ khách hàng ở lại rất cao, đặc biệt là Two Year gần như không có khách hàng rời đi. Điều này chỉ ra rằng các hợp đồng dài hạn hiệu quả hơn trong việc giữ chân khách hàng.

In [ ]:
# barplot of offer vs count of customer status
plt.figure(figsize=(10, 10))
sns.catplot(data=train_df, x='Offer', hue='Customer Status', kind='count')

Biểu đồ cột này so sánh số lượng khách hàng theo các loại ưu đãi (Offer) khác nhau. Offer B có số lượng khách hàng ở lại (màu xanh) cao nhất và tỷ lệ khách hàng rời đi thấp, cho thấy đây là ưu đãi hiệu quả nhất trong việc giữ chân khách hàng. Ngược lại, Offer E có số lượng khách hàng rời đi (màu cam) cao hơn số khách hàng ở lại, chỉ ra rằng đây là ưu đãi kém hiệu quả nhất. Offer A, C và D có tỷ lệ giữ chân khách hàng tốt, với số lượng khách hàng ở lại vượt trội so với số rời đi

## 4. Xử lý dữ liệu

### Xử lý thuộc tính địa lý

Nỗi bảng train với bảng dữ liệu về zip code và mật độ dân số

In [ ]:
def merge_zipcode(df):
    df['Zip Code'] = df['Zip Code'].astype(str)
    zip_df['zip'] = zip_df['zip'].astype(str)

    df_merged = pd.merge(df, zip_df, left_on='Zip Code', right_on='zip', how='left')

    unmatched = df_merged[df_merged['density'].isna()]
    print(f"Number of unmatched Zip codes: {len(unmatched)}")
    return df_merged

In [ ]:
# Concatenate geographical data -> population density
train_df = merge_zipcode(train_df)
test_df = merge_zipcode(test_df)

### Xử lý dữ liệu bị thiếu

In [ ]:
train_df.isnull().sum().sort_values(ascending=False)

In [ ]:
def process_missing_value(df):
    # process avg monthly long distance charges and multiple lines
    df['Avg Monthly Long Distance Charges'] = df['Avg Monthly Long Distance Charges'].fillna(0)
    df['Multiple Lines'] = df['Multiple Lines'].fillna("No")
    
    # process Internet service
    service_cols = ['Online Security', 'Online Backup', 'Device Protection Plan', 'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data']
    df[service_cols] = df[service_cols].fillna("No")
    df['Avg Monthly GB Download'] = df['Avg Monthly GB Download'].fillna(0)

    # Create 'No Offer' category (high absence percentage suggests that absence might be meaningful)
    df['Offer'] = df['Offer'].fillna('No Offer')

    df['density'] = df['density'].fillna(df['density'].median())

In [ ]:
process_missing_value(train_df)
process_missing_value(test_df)

In [ ]:
train_df.isnull().sum().sort_values(ascending=False)

### Hợp nhất các thuộc tính về dịch vụ Internet

In [ ]:
internet_service = [
    'Online Security',
    'Online Backup',
    'Device Protection Plan',
    'Premium Tech Support',
    'Streaming TV',
    'Streaming Movies',
    'Streaming Music',
    'Unlimited Data'
]

In [ ]:
def count_services(row):
    return sum(row[service] == 'Yes' for service in internet_service)

In [ ]:
train_df['Number of Services'] = train_df.apply(count_services, axis=1)
test_df['Number of Services'] = test_df.apply(count_services, axis=1)

### Bỏ đi những thuộc tính thừa

In [ ]:
def remove_unwanted_columns(df):
    unwanted_cols = ['Customer ID', 'Zip Code', 'City', 'Latitude', 'Longitude', 'Customer Status', 'Churn Reason', 'Churn Category', 'zip']
    # concatenate internet services
    for service in internet_service:
        unwanted_cols.append(service)
    return df.drop(columns=unwanted_cols)


In [ ]:
x_train_raw = remove_unwanted_columns(train_df)
y_train = train_df['Customer Status']

x_test_raw = remove_unwanted_columns(test_df)
y_test = test_df['Customer Status']

In [ ]:
print('training shape', x_train_raw.shape)
print('test shape', x_test_raw.shape)

In [ ]:
for att in x_train_raw.columns:
    print(att, x_train_raw[att].unique())

### Thuộc tính phân loại nhị phân

In [ ]:
def is_binary_attribute(series):
    unique_values = series.dropna().unique()
    return unique_values.size == 2

In [ ]:
binary_attributes = [col for col in x_train_raw.columns if is_binary_attribute(train_df[col])]
binary_attributes

In [ ]:
label_encoder = LabelEncoder()

for col in binary_attributes:
    x_train_raw[col] = label_encoder.fit_transform(x_train_raw[col])
    x_test_raw[col] = label_encoder.fit_transform(x_test_raw[col])

In [ ]:
x_train_raw[binary_attributes].head()

### Thuộc tính phân loại (> 2 giá trị)

In [ ]:
columns_to_onehot = ['Offer', 'Internet Type', 'Contract', 'Payment Method']#, 'Location Type']

In [ ]:
def encode_categorical_feature(df, columns):
    ohe = OneHotEncoder(sparse_output=False)
    encoded_cols = ohe.fit_transform(df[columns])
    new_cols_name = ohe.get_feature_names_out(columns)
    encoded_df = pd.DataFrame(encoded_cols, columns=new_cols_name, index=df.index)
    df = pd.concat([df.drop(columns=columns, axis=1), encoded_df], axis=1)
    return df

In [ ]:
x_train_raw = encode_categorical_feature(x_train_raw, columns_to_onehot)
x_test_raw = encode_categorical_feature(x_test_raw, columns_to_onehot)

In [ ]:
x_train_raw.head()

### Thuộc tính số nguyên

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

integral_attribute = ['Age', 'Number of Dependents', 'Number of Referrals', 'Tenure in Months', 'density', 'Number of Services']

x_train_raw[integral_attribute] = scaler.fit_transform(x_train_raw[integral_attribute])
x_test_raw[integral_attribute] = scaler.fit_transform(x_test_raw[integral_attribute])

In [ ]:
x_train_raw[integral_attribute].head()

### Thuộc tính số thực

In [ ]:
float_attribute = ['Avg Monthly Long Distance Charges', 'Avg Monthly GB Download', 'Monthly Charge', 'Total Charges', 'Total Refunds','Total Extra Data Charges', 'Total Long Distance Charges', 'Total Revenue']

scaler = MinMaxScaler(feature_range=(0, 1))

train_df[float_attribute] = scaler.fit_transform(train_df[float_attribute])
test_df[float_attribute] = scaler.fit_transform(test_df[float_attribute])

In [ ]:
train_df[float_attribute].head()

### SMOTE

Do số lượng khách hàng rời đi trong tập train rất ít sơ với khách hàng ở lại nên ta sẽ dùng SMOTE để có thể cân bằng số lượng rời đi và ở lại trong tập train, điều này giúp các mô hình ở phần sau có thêm data và do đó hiệu suất có thể được nâng cao hơn

In [ ]:
smote = SMOTE(random_state=42)
x_train_raw_balanced, y_train_balanced = smote.fit_resample(x_train_raw, y_train)

print("Before SMOTE: ", y_train.value_counts())
print("After SMOTE: ", y_train_balanced.value_counts())

## 6. Lựa chọn thuộc tính

In [ ]:
x_train_raw.columns
print(x_train_raw.columns)

In [ ]:
def plot_feature_importance(importance, names, model_type):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names': feature_names, 'feature_importance': feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False, inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10, 20))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

### Correlation

In [ ]:
def get_corr_features(df):
    """
    Get features with correlation > 0.1
    :param df: dataframe with numeric features
    :return:
    """
    corr_feat = list(df.corr()[abs(df.corr()['Customer Status']) > 0.1].index)
    corr_feat.remove('Customer Status')
    return corr_feat

In [ ]:
train_numeric = pd.concat([x_train_raw, y_train], axis=1)
corr_feat = get_corr_features(train_numeric)

In [ ]:
print(corr_feat)

### Annova

In [ ]:
def get_annova_features(df):
    """
    Get features with ANNOVA > 0.1
    :param df: dataframe with numeric features
    :return:
    """
    annova_scores, _ = f_classif(np.abs(df.values), y_train)
    annova_feats_score = list(zip(df.columns, annova_scores))
    annova_feats_score.sort(key=lambda x: -x[1])
    # choose top 20 features
    annova_feats = [i[0] for i in annova_feats_score[:20]]
    annova_feats.remove('Customer Status')
    plot_feature_importance(
        [i[1] for i in annova_feats_score],
        [i[0] for i in annova_feats_score],
        'Annova'
    )
    return annova_feats

In [ ]:
annova_feats = get_annova_features(train_numeric)

### Phương pháp sử dụng mô hình

In [ ]:
def get_lgbm_features(df):
    model_fi = lgb.LGBMClassifier()
    model_fi.fit(x_train_raw, y_train)
    lgbm_scores = model_fi.feature_importances_

    lgbm_feat_scores = list(zip(df.columns, lgbm_scores))
    lgbm_feat_scores.sort(key=lambda x: -x[1])

    # choose top 20 features
    lgbm_feats = [i[0] for i in lgbm_feat_scores[:20]]

    plot_feature_importance(
        [i[1] for i in lgbm_feat_scores],
        [i[0] for i in lgbm_feat_scores],
        'LightGBM'
    )

    return lgbm_feats

In [ ]:
lgbm_feats = get_lgbm_features(train_numeric)

## 7. Mô hình phân lớp

### Hàm đánh giá kết quả

In [ ]:
# Evaluate model accuracy
def evaluate(model, x_test, y_test):
    pred = model.predict(x_test)
    acc = accuracy_score(y_test, pred)
    print("ACC: ", acc)
    print(classification_report(y_test,pred))


# Visualize confusion matrix
def visualize_result(model, x_test, y_test):
    y_pred = model.predict(x_test)
    sns.heatmap(confusion_matrix(y_test,y_pred),cmap='Blues',annot=True,linewidths=2,linecolor='white')

### Sử dụng toàn bộ feature

In [ ]:
# Scaling data using standard scaler

scaler = StandardScaler()
scaler.fit(x_train_raw_balanced)
x_train_scaled = scaler.transform(x_train_raw_balanced)
x_test_scaled = scaler.transform(x_test_raw)

print('training shape: ', x_train_scaled.shape)
print('testing shape: ', x_test_scaled.shape)

### Decision Tree

In [ ]:
def find_max_depth(x_train, y_train, x_test, y_test):
    min_err_rate = 1
    res = 0
    for i in range(5, 20):
        tree = DecisionTreeClassifier(max_depth=i)
        tree.fit(x_train, y_train)
        pred_i = tree.predict(x_test)
        err_rate = np.mean((pred_i != y_test) & (y_test == 1))
        if err_rate < min_err_rate:
            min_err_rate = err_rate
            res = i
    
    print(f"min error rate: {min_err_rate} with depth: {res}")
    return res

In [ ]:
def decision_tree_classifier(x_train, y_train, x_test, y_test):
    depth = find_max_depth(x_train, y_train, x_test, y_test)
    tree = DecisionTreeClassifier(max_depth=depth, criterion='log_loss')
    tree.fit(x_train, y_train)
    evaluate(tree, x_test, y_test)
    visualize_result(tree, x_test, y_test)

In [ ]:
decision_tree_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Kết quả của mô hình Decision Tree Classifier cho thấy một hiệu suất tổng thể khá tốt với độ chính xác 80.36%. Tuy nhiên, khi xem xét kỹ hơn, ta thấy có sự chênh lệch đáng kể giữa hiệu suất của hai lớp. Lớp 0 có hiệu suất tốt với precision, recall và f1-score đều trên 0.82, trong khi lớp thiểu số (lớp 1) có hiệu suất thấp hơn đáng kể, đặc biệt là về precision (0.58). Điều này phản ánh một trong những hạn chế của Decision Tree khi đối mặt với dữ liệu không cân bằng. Cây quyết định có xu hướng ưu tiên các đặc trưng phổ biến, dẫn đến việc dự đoán tốt hơn cho lớp đa số nhưng kém chính xác hơn cho lớp thiểu số. Tuy nhiên, điểm đáng chú ý là recall của lớp thiểu số (0.77) vẫn ở mức khá, cho thấy mô hình có khả năng nhận diện được phần lớn các mẫu thuộc lớp này. Sự chênh lệch giữa precision và recall của lớp thiểu số gợi ý rằng mô hình có xu hướng phân loại nhiều mẫu vào lớp thiểu số, dẫn đến tỷ lệ dương tính giả cao. Nguyên nhân có thể là do overfitting, thiếu tinh chỉnh hyperparameter.

### KNN

In [ ]:
def find_k_neighbors(x_train, y_train, x_test, y_test):
    res = 0
    min_err = 1
    for i in range(1, 50):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(x_train, y_train)
        pred_i = knn.predict(x_test)
        err_rate = np.mean((pred_i != y_test) & (y_test == 1))
        if err_rate < min_err:
            min_err = err_rate
            res = i
            
    print('k =', res, 'error rate =', min_err)
    return res

In [ ]:
def knn_classifier(x_train, y_train, x_test, y_test):
    k = find_k_neighbors(x_train, y_train, x_test, y_test)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    evaluate(knn, x_test, y_test)
    visualize_result(knn, x_test, y_test)

In [ ]:
knn_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Kết quả của mô hình KNN cho thấy một độ chính xác tổng thể ở mức trung bình với 74.66%. Điều đáng chú ý là sự khác biệt rõ rệt giữa hiệu suất của hai lớp. Lớp đa số (lớp 0) có precision rất cao (0.95) nhưng recall thấp hơn (0.70), trong khi lớp thiểu số (lớp 1) có xu hướng ngược lại với recall cao (0.89) nhưng precision thấp (0.50). Điều này cho thấy mô hình KNN có xu hướng phân loại nhiều mẫu vào lớp thiểu số, dẫn đến tỷ lệ dương tính giả cao cho lớp này. Đặc điểm này của KNN có thể là do việc chọn k = 45, một giá trị khá lớn so với kích thước của lớp thiểu số (387 mẫu). Với k lớn, mô hình có xu hướng ưu tiên lớp đa số trong không gian đặc trưng, nhưng đồng thời cũng "bắt" được nhiều mẫu thuộc lớp thiểu số. Điều này giải thích cho recall cao của lớp thiểu số nhưng precision thấp. Ngược lại, lớp đa số có precision cao nhưng recall thấp hơn, cho thấy khi mô hình dự đoán một mẫu thuộc lớp đa số, nó thường chính xác, nhưng có nhiều mẫu thực sự thuộc lớp đa số bị phân loại nhầm sang lớp thiểu số.

### SVM

In [ ]:
def choose_C(x_train, y_train, x_test, y_test):
    min_err_rate = 1
    res = 0
    for i in range(1, 10):
        svc = SVC(C=i, gamma='auto', kernel='rbf')
        svc.fit(x_train, y_train)
        pred_i = svc.predict(x_test)
        err_rate = np.mean((pred_i != y_test) & (y_test == 1))
        if err_rate < min_err_rate:
            min_err_rate = err_rate
            res = i

    print(f"min error rate: {min_err_rate} with C: {res}")
    return res

In [ ]:
def SVC_classifier(x_train, y_train, x_test, y_test):
    c = choose_C(x_train, y_train, x_test, y_test)
    sup = SVC(C = c, gamma = 'auto', kernel = 'rbf')
    sup.fit(x_train, y_train)
    evaluate(sup, x_test, y_test)
    visualize_result(sup, x_test, y_test)

In [ ]:
SVC_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Kết quả của mô hình SVC cho thấy một hiệu suất tổng thể khá tốt với độ chính xác 82.76%. Đây là một cải thiện đáng kể so với mô hình KNN trước đó và cũng nhỉnh hơn một chút so với Decision Tree. Khi xem xét chi tiết, ta thấy có sự chênh lệch giữa hiệu suất của hai lớp, nhưng không quá lớn như trong trường hợp của KNN. Lớp đa số (lớp 0) có precision rất cao (0.92) và recall khá tốt (0.84), cho thấy mô hình rất hiệu quả trong việc nhận diện và phân loại chính xác các mẫu thuộc lớp này. Đối với lớp thiểu số (lớp 1), mô hình đạt được recall khá cao (0.78), cao hơn so với Decision Tree, nhưng precision vẫn ở mức thấp (0.62), tương tự như Decision Tree. Điều này cho thấy SVC có khả năng phát hiện tốt các mẫu thuộc lớp thiểu số, nhưng vẫn có xu hướng phân loại nhầm một số mẫu của lớp đa số vào lớp thiểu số. Sự cân bằng tốt hơn giữa precision và recall của cả hai lớp đã dẫn đến f1-score cao hơn so với các mô hình trước đó, đặc biệt là đối với lớp thiểu số (0.69). Điều này phản ánh một trong những ưu điểm của SVC trong việc xử lý dữ liệu không cân bằng, có thể là nhờ khả năng tạo ra một đường biên quyết định phức tạp và linh hoạt trong không gian đặc trưng.

### Logistic Regression

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test):
    logr = LogisticRegression(C=1, max_iter=200, solver='liblinear')
    logr.fit(x_train, y_train)
    evaluate(logr, x_test, y_test)
    visualize_result(logr, x_test, y_test)

In [ ]:
logistic_regression(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Kết quả của mô hình Hồi quy Logistic cho thấy một hiệu suất tổng thể khá tốt với độ chính xác 80.23%. Mặc dù không cao bằng SVC, nhưng vẫn tốt hơn KNN và gần với Decision Tree. Khi xem xét chi tiết, ta thấy có sự khác biệt đáng kể giữa hiệu suất của hai lớp. Lớp đa số (lớp 0) có precision rất cao (0.94), cao nhất trong số các mô hình đã xem xét, nhưng recall thấp hơn (0.78). Điều này cho thấy khi mô hình dự đoán một mẫu thuộc lớp 0, nó rất có khả năng chính xác, nhưng mô hình cũng bỏ sót một số mẫu thực sự thuộc lớp này. Đối với lớp thiểu số (lớp 1), ta thấy một xu hướng ngược lại: recall cao (0.86), thậm chí cao nhất trong số các mô hình đã xem xét, nhưng precision thấp (0.57). Điều này ngụ ý rằng mô hình có khả năng phát hiện tốt các mẫu thuộc lớp thiểu số, nhưng cũng có xu hướng phân loại nhầm nhiều mẫu của lớp đa số vào lớp thiểu số. Sự chênh lệch lớn giữa precision và recall của cả hai lớp cho thấy mô hình Hồi quy Logistic đang gặp khó khăn trong việc cân bằng giữa hai lớp không cân bằng. Điều này có thể là do bản chất tuyến tính của mô hình Hồi quy Logistic, khiến nó khó tạo ra một đường biên quyết định phức tạp để phân tách tốt hai lớp trong không gian đặc trưng. Tuy nhiên, f1-score của lớp thiểu số (0.68) vẫn khá tốt, chỉ thấp hơn một chút so với SVC.

### Random Forest Classifier

In [ ]:
def find_n_estimators(x_train, y_train, x_test, y_test):
    min_err_rate = 1
    res = 0
    for i in range(1, 20):
        rfe = RandomForestClassifier(n_estimators=i*10) 
        rfe.fit(x_train, y_train) 
        errpred = rfe.predict(x_test) 
        err = np.mean((errpred != y_test) & (y_test == 1))
        if err < min_err_rate:
            min_err_rate = err
            res = i*10
            
    print(f"min error rate: {min_err_rate} with estimators: {res}")
    return res

In [ ]:
def random_forest_classifier(x_train, y_train, x_test, y_test):
    estimators = find_n_estimators(x_train, y_train, x_test, y_test)
    rfe = RandomForestClassifier(n_estimators=estimators, max_depth=10)
    rfe.fit(x_train, y_train)
    evaluate(rfe, x_test, y_test)
    visualize_result(rfe, x_test, y_test)

In [ ]:
random_forest_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Kết quả của mô hình Random Forest cho thấy hiệu suất tổng thể rất tốt với độ chính xác 84.45%, cao nhất trong số các phương pháp đã xem xét. Điều này phản ánh một trong những ưu điểm chính của Random Forest - khả năng xử lý hiệu quả các bộ dữ liệu phức tạp và không cân bằng. Khi phân tích chi tiết, ta thấy có sự cân bằng tốt giữa hiệu suất của hai lớp, điều mà các phương pháp khác chưa đạt được.
Đối với lớp đa số (lớp 0), mô hình đạt được cả precision (0.92) và recall (0.87) cao, dẫn đến f1-score rất tốt (0.89). Điều này cho thấy Random Forest rất hiệu quả trong việc nhận diện và phân loại chính xác các mẫu thuộc lớp đa số, với rất ít trường hợp bỏ sót hoặc phân loại sai.
Đối với lớp thiểu số (lớp 1), mặc dù hiệu suất thấp hơn so với lớp đa số, nhưng vẫn đạt được sự cân bằng tốt giữa precision (0.67) và recall (0.76), dẫn đến f1-score khá cao (0.71). Đây là f1-score cao nhất cho lớp thiểu số trong số các phương pháp đã xem xét, cho thấy Random Forest xử lý tốt vấn đề không cân bằng dữ liệu.
Sự cân bằng tốt giữa precision và recall của cả hai lớp phản ánh một trong những ưu điểm chính của Random Forest: khả năng tạo ra một tập hợp các cây quyết định đa dạng, giúp mô hình có thể nắm bắt được các mẫu phức tạp trong dữ liệu. Điều này đặc biệt hữu ích trong trường hợp dữ liệu không cân bằng, nơi một số cây có thể chuyên biệt hóa trong việc nhận diện các mẫu thuộc lớp thiểu số.

### Sử dụng feature tương quan cao

In [ ]:
x_train_raw_feat = x_train_raw_balanced[corr_feat].copy(deep=True)
x_test_raw_feat = x_test_raw[corr_feat].copy(deep=True)

In [ ]:
scaler.fit(x_train_raw_feat)
x_train_scaled = scaler.transform(x_train_raw_feat)
x_test_scaled = scaler.transform(x_test_raw_feat)

In [ ]:
random_forest_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

### Sử dụng feature anova

In [ ]:
x_train_raw_feat2 = x_train_raw_balanced[annova_feats].copy(deep=True)
x_test_raw_feat2 = x_test_raw[annova_feats].copy(deep=True)

In [ ]:
scaler.fit(x_train_raw_feat2)
x_train_scaled = scaler.transform(x_train_raw_feat2)
x_test_scaled = scaler.transform(x_test_raw_feat2)

In [ ]:
random_forest_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

### Sử dụng feature chọn từ mô hình

In [ ]:
x_train_raw_feat3 = x_train_raw_balanced[lgbm_feats].copy(deep=True)
x_test_raw_feat3 = x_test_raw[lgbm_feats].copy(deep=True)

scaler.fit(x_train_raw_feat3)
x_train_scaled = scaler.transform(x_train_raw_feat3)
x_test_scaled = scaler.transform(x_test_raw_feat3)

In [ ]:
random_forest_classifier(x_train_scaled, y_train_balanced, x_test_scaled, y_test)

Việc lựa chọn thuộc tính giúp giảm chiều dữ liệu và không làm thay đổi kết quả cuối cùng quá đáng kể.
Việc sử dụng các biến tương quan mạnh với nhãn cho hiệu quả không quá cao, vì tương quan này chỉ là mối quan hệ giữa các thuộc tính và nhãn 1 cách độc lập (đơn biến). Điều này không hiệu quả vì thường kết hợp đa biến cho độ chính xác tốt hơn.
Phương pháp anova và dùng mô hình cũng không thay đổi hiệu suất một cách đáng kể, điều này thể hiện các đặc trưng quan trọng đã được giữ lại, do đó không ảnh hưởng nhiều đến kết quả cuối cùng.

# 8. Học sâu với mô hình mạng nơ-ron nhân tạo ANN(Artificial Neural Network)

## Xây dựng mô hình

In [ ]:
# Xây dựng mô hình ANN
model = Sequential()
model.add(Input(shape=(x_train_scaled.shape[1],)))  # Xác định kích thước đầu vào
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile mô hình
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Huấn luyện mô hình

In [ ]:
# Huấn luyện mô hình
history = model.fit(x_train_scaled, y_train_balanced, epochs=50, batch_size=32, validation_data=(x_test_scaled, y_test))

## Đánh giá mô hình

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(x_test_scaled, y_test)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

# Dự đoán trên dữ liệu kiểm tra
predictions = model.predict(x_test_scaled)
predicted_classes = (predictions > 0.5).astype("int32")

acc = accuracy_score(y_test, predicted_classes)
print("ACC: ", acc)
print(classification_report(y_test,predicted_classes))

# Lịch sử huấn luyện mô hình
history_dict = history.history

## Biểu đồ mất mát và biểu đồ độ chính xác trong quá trình huấn luyện mô hình

In [ ]:
# Vẽ biểu đồ mất mát (loss) trong quá trình huấn luyện
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Vẽ biểu đồ độ chính xác (accuracy) trong quá trình huấn luyện
plt.subplot(1, 2, 2)
plt.plot(history_dict['accuracy'], label='Training Accuracy')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

## Biểu đồ kết quả dự đoán dựa trên dữ liệu kiểm tra

In [ ]:
# Dự đoán trên dữ liệu kiểm tra và vẽ biểu đồ kết quả
plt.figure(figsize=(6, 4))
plt.hist(predicted_classes, bins=np.arange(0, 2) - 0.5, edgecolor='black', alpha=0.7)
plt.title('Histogram of Predictions on Test Data')
plt.xlabel('Predicted Class')
plt.ylabel('Frequency')
plt.xticks([0, 1], ['Class 0', 'Class 1'])
plt.grid(True)

plt.show()

## Kết luận

### Kết quả huấn luyện
Accuracy và Loss trong tập huấn luyện và tập kiểm tra: Trong quá trình huấn luyện, độ chính xác (accuracy) trên tập huấn luyện tăng dần từ khoảng 60% lên hơn 85%. Độ chính xác trên tập kiểm tra cũng tăng dần, đạt đến khoảng 82-83%. Loss (mất mát) giảm dần, cho thấy mô hình đang học và cải thiện qua từng epoch.
Validation Accuracy và Loss: Độ chính xác trên tập kiểm tra dao động quanh mức 82-83%. Loss trên tập kiểm tra giảm dần, nhưng sau đó có xu hướng ổn định và thậm chí có tăng nhẹ.
### Kết quả đánh giá mô hình
Accuracy trên tập kiểm tra là 83.71%.
Loss trên tập kiểm tra là 0.3748.
Validation Accuracy là: 83.09%. 
Độ chính xác của mô hình trên tập huấn luyện và tập kiểm tra đều cao, cho thấy mô hình học tốt từ dữ liệu và có khả năng tổng quát tốt.
Độ chính xác trên tập kiểm tra khá ổn định qua các epoch cuối cùng, điều này cho thấy mô hình không bị overfitting quá nhiều.
Loss trên tập kiểm tra có xu hướng ổn định sau một thời gian giảm, điều này cho thấy mô hình đã đạt đến mức tối ưu nhất định.

### So sánh với phương pháp trên
Phân tích kết quả của mô hình Deep Learning (ANN) cho bài toán dự đoán khách hàng rời bỏ cho thấy hiệu suất ấn tượng với độ chính xác tổng thể 82.18%. Mô hình này đứng thứ ba trong số các phương pháp đã xem xét, chỉ sau Random Forest và SVC, nhưng vượt trội hơn Logistic Regression và KNN. Đối với lớp đa số (khách hàng ở lại), ANN thể hiện hiệu suất rất tốt với precision 0.92, recall 0.84 và F1-score 0.88, gần như ngang bằng với SVC. Đáng chú ý là hiệu suất của mô hình đối với lớp thiểu số (khách hàng rời bỏ), đạt được recall cao 0.78, cho thấy khả năng tốt trong việc nhận diện khách hàng có nguy cơ rời bỏ. F1-score 0.69 cho lớp này tương đương với SVC và chỉ kém hơn một chút so với Random Forest. Sự cân bằng tốt giữa precision và recall cho cả hai lớp gợi ý rằng AUC-ROC của mô hình có thể khá cao, mặc dù không được cung cấp trực tiếp. So với các phương pháp Machine Learning cổ điển, ANN thể hiện ưu điểm trong việc học các mối quan hệ phi tuyến phức tạp mà không cần nhiều feature engineering. Tuy nhiên, Random Forest vẫn cho kết quả tổng thể tốt hơn một chút, có thể do khả năng xử lý tự nhiên đối với dữ liệu không cân bằng và khả năng chống overfitting. Nhìn chung, mô hình ANN là một lựa chọn rất tốt cho bài toán churn prediction này, cung cấp sự cân bằng hiệu quả giữa khả năng nhận diện khách hàng rời bỏ và độ chính xác của những dự đoán đó.